In [17]:
import random
import socket
import struct
import numpy as np
import copy
import hashlib
import intervals
from pprint import pprint

In [39]:
numberOfBitsOfHash = 3
# numberOfBitsOfHash=3
# numberOfNodes=100
# periodToCorrectFingerTable = 10

In [40]:
def contains(key1,lowerBound,upperBound,numberOfBitsOfHash,flag):
    if lowerBound==upperBound:
        return 1
    if(flag==1): #Closed Interval
        if key1==lowerBound or key1==upperBound:
            return 1
    elif(flag==2): #Open Interval
        if key1==lowerBound or key1==upperBound:
            return 0
    elif(flag==3): #open closed interval
        if key1==lowerBound:
            return 0
        elif key1==upperBound:
            return 1
    elif(flag==4): #closed open interval
        if key1==lowerBound:
            return 1
        elif key1==upperBound:
            return 0
    

    if lowerBound<upperBound:
        diff = upperBound - lowerBound
        temp1 = key1 - lowerBound
        temp2 = upperBound-key1
        if temp1<diff and temp2<diff:
            return 1
        else:
            return 0
    elif lowerBound>upperBound:
        temp=lowerBound
        lowerBound=upperBound
        upperBound=temp
        diff = upperBound - lowerBound
        temp1 = key1 - lowerBound
        temp2 = upperBound-key1
        if temp1<diff and temp2<diff:
            return 0
        else:
            return 1
        
        
        
def initializeFingerTableOfFirstNode(firstNode):
    start_list=[]
    interval_list=[]
    successorList=[]
    for k in range(1,numberOfBitsOfHash+1):
        start=(firstNode.nodeId+pow(2,k-1))%(pow(2,numberOfBitsOfHash))
        end=(firstNode.nodeId+pow(2,k)-1)%(pow(2,numberOfBitsOfHash))
        start_list.append(start)
        successorList.append(firstNode)    
    fingerTable = np.rec.fromarrays((start_list,successorList ), names=('start','node'))  

    return fingerTable

def createChordRing(firstNode):
    firstNode.successor=firstNode
    firstNode.fingerTable=initializeFingerTableOfFirstNode(firstNode)
    firstNode.predecessor = firstNode
    return firstNode


def initializeNewNode(newNodeId):
    newNode = Node(newNodeId)
    newNode.successor = masterNode.findSuccessor(newNode.nodeId)
    newNode.fingerTable = initializeFingerTableOfFirstNode(newNode)
    return newNode

def printNodeAttributes(nodeValue):
    print("Node Id:",nodeValue.nodeId)
    print("Node Successor:",nodeValue.successor.nodeId)
    print("Node Predecessor:",nodeValue.predecessor.nodeId)
    for i in nodeValue.fingerTable:
        print("Start: ",i.start," Succ:",i.node.nodeId)
     
        
def addFirstTwoNodes(firstNode,secondNode):
    for index,i in enumerate(firstNode.fingerTable):
        if contains(i.start,firstNode.nodeId,secondNode.nodeId,numberOfBitsOfHash,1):
            firstNode.fingerTable[index].node=secondNode

    for index,i in enumerate(secondNode.fingerTable):
        if contains(i.start,secondNode.nodeId,firstNode.nodeId,numberOfBitsOfHash,1):
            secondNode.fingerTable[index].node=firstNode
    
    firstNode.successor=secondNode
    firstNode.predecessor=secondNode
    secondNode.successor=firstNode
    secondNode.predecessor=firstNode
    return (firstNode,secondNode)

In [41]:
class Node:
    def __init__(self, nodeId):
        self.nodeId = nodeId
        self.fingerTable = None
        self.predecessor=None
        self.successor=[nodeId]
        self.data={}
    
    def findClosestPreceedingFinger(self,id):
        for i in range(numberOfBitsOfHash-1,-1,-1):
            if(contains(self.fingerTable[i].node.nodeId,self.nodeId,id,numberOfBitsOfHash,2)):
                return self.fingerTable[i].node
        return self   
    
    def findPredecessor(self,id):
        idCopy=self
        while (not contains(id,idCopy.nodeId,idCopy.successor.nodeId,numberOfBitsOfHash,3)):
            idCopy=idCopy.findClosestPreceedingFinger(id)
        return idCopy
    
    def findSuccessor(self,id):
        predecessor = self.findPredecessor(id)
        return predecessor.successor
    
    #Joining a node
    def updateFingerTable(self,node,i):
        if self.nodeId==node.nodeId:
            return
        if contains(node.nodeId,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,2):
            self.fingerTable[i].node = node
            p = self.predecessor
            p.updateFingerTable(node,i)

    def updateOthers(self):
        for i in range(1,numberOfBitsOfHash+1):
            pred = self.findPredecessor((self.nodeId - pow(2,i-1))%pow(2,numberOfBitsOfHash))
            pred.updateFingerTable(self,i-1)

    def initFingerTable(self,existingNode):
        self.fingerTable[0].node = existingNode.findSuccessor(self.fingerTable[0].start)
        self.successor = self.fingerTable[0].node
        self.predecessor = self.successor.predecessor
        self.successor.predecessor = self
        self.predecessor.successor = self
        for i in range(0,numberOfBitsOfHash-1):
            if contains(self.fingerTable[i+1].start,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,4):
                self.fingerTable[i+1].node = self.fingerTable[i].node
            else:
                self.fingerTable[i+1].node = existingNode.findSuccessor(self.fingerTable[i+1].start)

    def join(self,existingNode):
        self.initFingerTable(existingNode)
        self.updateOthers()
        
    #Lookup Query
    def lookup(self,key):
        requiredNode = self.findSuccessor(key)
        return requiredNode
    
    #Node Failure
    def updateFingerTableOnFailure(self,node,i):
        print("Hyioyy")
        if(node.nodeId==self.fingerTable[i].node.nodeId):
            print("Hiii")
            self.fingerTable[i].node=node.successor
        elif contains(node.nodeId,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,2):
            print("yo")
            self.fingerTable[i].node = node.successor
            p = self.predecessor
            p.updateFingerTableOnFailure(node,i)

    def updateOthersOnFailure(self):
        for i in range(1,numberOfBitsOfHash+1):
#             print(":h")
            pred = self.findPredecessor((self.nodeId - pow(2,i-1))%pow(2,numberOfBitsOfHash))
            print(pred.nodeId)
            pred.updateFingerTableOnFailure(self,i-1)
            
    def nodeFailure(self):
        self.updateOthersOnFailure()
        self.predecessor.successor=self.successor
        self.successor.predecessor=self.predecessor

    
    #Stabilization of Network
    
    def notify(self,origNode):
        if self.predecessor or contains(origNode.nodeId,self.predecessor.nodeId,self.nodeId,numberOfBitsOfHash,2):
            self.predecessor=origNode
            
    def stabilize(self):
        x = self.successor.predecessor
        if contains(x.nodeId,self.nodeId,self.successor.nodeId,numberOfBitsOfHash,2):
            self.successor=x
        self.successor.notify(self)

    
    def fixFingerTables(self):
        for i in range(0,numberOfBitsOfHash):
            self.fingerTable[i].node = self.findSuccessor(self.fingerTable[i].start)
            
        return self
            
    #Adding data to a node
    def addData(self,key,value):
        nodeOfData = self.findSuccessor(key)
        nodeOfData.data[key]=value
            

In [42]:
firstNode = Node(0)
firstNode = createChordRing(firstNode)
secondNode=initializeNewNode(2)
thirdNode=initializeNewNode(5)
fourthNode=initializeNewNode(7)
nodeDict={0:firstNode,2:secondNode,5:thirdNode}
for index,i in enumerate(firstNode.fingerTable):
    if contains(i.start,firstNode.nodeId,secondNode.nodeId,numberOfBitsOfHash,1):
        firstNode.fingerTable[index].node=secondNode

for index,i in enumerate(secondNode.fingerTable):
    if contains(i.start,secondNode.nodeId,firstNode.nodeId,numberOfBitsOfHash,1):
        secondNode.fingerTable[index].node=firstNode
print("----------Initializing 2 Nodes(0,2)-------------")        
firstNode.successor=secondNode
firstNode.predecessor=secondNode
secondNode.successor=firstNode
secondNode.predecessor=firstNode
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("----------Adding 1 node(5)-------------")        
thirdNode.join(firstNode)
# thirdNode.fixFingerTables()
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("----------Adding 1 node(7)-------------")        
fourthNode.join(firstNode)
# thirdNode.fixFingerTables()
# fourthNode.fixFingerTables()
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("--------------")
printNodeAttributes(fourthNode)
# print("----------Removing 1 node(5)-------------")
# thirdNode.nodeFailure()
# # # del thirdNode
# printNodeAttributes(firstNode)
# print("--------------")
# printNodeAttributes(secondNode)
# print("--------------")
# printNodeAttributes(fourthNode)
# print("--------------")

----------Initializing 2 Nodes(0,2)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 2
Start:  1  Succ: 2
Start:  2  Succ: 2
Start:  4  Succ: 0
--------------
Node Id: 2
Node Successor: 0
Node Predecessor: 0
Start:  3  Succ: 0
Start:  4  Succ: 0
Start:  6  Succ: 0
----------Adding 1 node(5)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 5
Start:  1  Succ: 2
Start:  2  Succ: 2
Start:  4  Succ: 5
--------------
Node Id: 2
Node Successor: 5
Node Predecessor: 0
Start:  3  Succ: 5
Start:  4  Succ: 5
Start:  6  Succ: 0
--------------
Node Id: 5
Node Successor: 0
Node Predecessor: 2
Start:  6  Succ: 0
Start:  7  Succ: 0
Start:  1  Succ: 2
----------Adding 1 node(7)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 7
Start:  1  Succ: 2
Start:  2  Succ: 2
Start:  4  Succ: 5
--------------
Node Id: 2
Node Successor: 5
Node Predecessor: 0
Start:  3  Succ: 5
Start:  4  Succ: 5
Start:  6  Succ: 7
--------------
Node Id: 5
Node Successor: 7
Node Predecessor: 2
Start

In [30]:
# valueToAdd = "my name is pratik"
# key = 6
# firstNode.addData(key,valueToAdd)

In [31]:
#Generate IP Addresses
all_ip_address=[]
for i in range(0,numberOfNodes):
    temp = socket.inet_ntoa(struct.pack('>I', random.randint(0x3cffffff, 0xffffffff)))
    if(temp not in all_ip_address):
        all_ip_address.append(temp)
    else:
        i-=1

#Calculating SHA-1 (160 bit) hash of IP Address of a node
hashSet = {}
for ip in all_ip_address:
    hashSet[ip]=int(hashlib.sha1(ip.encode('utf-8')).hexdigest(),16)

sortedHashSet = {k: v for k, v in sorted(hashSet.items(), key=lambda item: item[1])}
sortedKeys = list(sortedHashSet.keys())

masterNode = Node(sortedHashSet[sortedKeys[0]])
masterNode = createChordRing(masterNode)
secondNode=initializeNewNode(sortedHashSet[sortedKeys[1]])
(masterNode,secondNode)=addFirstTwoNodes(masterNode,secondNode)
nodeListDictionary={}
nodeListDictionary[sortedHashSet[sortedKeys[0]]]=masterNode
nodeListDictionary[sortedHashSet[sortedKeys[1]]]=secondNode

flag = 0 
tempList=[]
for i in range(2,numberOfNodes):
#     if flag==periodToCorrectFingerTable:
#         flag = 0
#         for j in tempList:
#             j.fixFingerTables()
#         tempList
    tempNode = initializeNewNode(sortedHashSet[sortedKeys[i]])
    tempNode.join(masterNode)
    tempList.append(tempNode)
    flag+=1
    nodeListDictionary[sortedHashSet[sortedKeys[i]]]=tempNode

#Fixing Finger Tables
for key,value in nodeListDictionary.items():
    nodeListDictionary[key]=value.fixFingerTables()


In [32]:
#Add Random Data
data = "my name is pratik"
keyData = int(hashlib.sha1(data.encode('utf-8')).hexdigest(),16)
nodeListDictionary

{13211937154097692222366146132253959956382767542: <__main__.Node at 0x7f1afbfa76d0>,
 51313466374555249138493444116729913906771198408: <__main__.Node at 0x7f1afc3898d0>,
 72509014145712803423783525702061652248481215014: <__main__.Node at 0x7f1afc431b50>,
 82687762006990260279317043652912695114663410540: <__main__.Node at 0x7f1afbfae890>,
 103045456899040198942874757303471967251151375532: <__main__.Node at 0x7f1afc431090>,
 115343247062121081336480882724798053791951555311: <__main__.Node at 0x7f1afc28f7d0>,
 143262844345319232963801867082037713669633673038: <__main__.Node at 0x7f1afc5c0cd0>,
 148419352376252915079680989271887283194672394785: <__main__.Node at 0x7f1afbfb6e50>,
 158842045896446276776497064297964385027356399488: <__main__.Node at 0x7f1afbfb6e90>,
 161241063231438743458688007664365970479022729067: <__main__.Node at 0x7f1afbfac790>,
 178020133970659256894494202327895287850048701315: <__main__.Node at 0x7f1afc5e1090>,
 215840814653375739604449604320203365967043651486: <__main

In [ ]:
numberOfDataPoints=10000

In [ ]:
dataStored = []

In [ ]:
#Storing 10000 dataPoints
dataStored = []
i=0
while(1):
    print(i)
    if(i==10000):
        break
    temp = random.randint(10,200000)
    if temp in dataStored:
        i-=1
        continue
    data = str(temp)
    keyData = int(hashlib.sha1(data.encode('utf-8')).hexdigest(),16)
    masterNode.addData(keyData,data)
    dataStored.append(temp)
    i+=1
    

In [ ]:
random.randint(10,20000)

In [ ]:
len(dataStored)

In [ ]:
dataToLook=str(dataStored[0])
key = int(hashlib.sha1(dataToLook.encode('utf-8')).hexdigest(),16)

In [ ]:
masterNode.lookup(key).nodeI

In [ ]:
dataToLook

In [ ]:
key